In [ ]:
import pandas as pd
from pathlib import Path
import sys

# Adiciona o diretório 'Code' ao path para encontrar o DataUtils
# Assumindo que o notebook está na raiz do projeto e o script em 'Code'
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

from DataUtils import AnalisadorBancario

# --- Configurações ---
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
output_dir = Path('..').resolve() / 'Output'

# --- Inicializar o Analisador ---
# A classe carrega todos os dados na inicialização.
analisador = AnalisadorBancario(diretorio_output=str(output_dir))

In [ ]:
# Objetivo: Pegar o Patrimônio Líquido e o Ativo Total do Itaú em dez/2023.
# Usaremos o COSIF Prudencial, que é o padrão.
# Para balanços, o documento 4060 é geralmente o balancete e 4066 o balanço. Vamos pegar de ambos.

contas_interesse = ['ATIVO REALIZÁVEL', 'Caixa']
data_interesse = 202412

dados_itau_cosif = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=contas_interesse,
    datas=data_interesse,
    documentos=[4060, 4066] # Exemplo de filtro por documento
)

print("--- Dados COSIF para Bco do Brasil ---")
display(dados_itau_cosif)

In [ ]:
# Objetivo: Tentar pegar o mesmo dado para a Nu Financeira.
# A classe deve traduzir '30680829' para os identificadores corretos.

dados_nubank_cosif = analisador.get_dados_cosif(
    identificador='18236120', # CNPJ da Nu Financeira
    contas=contas_interesse,
    datas=data_interesse
)

print("\n--- Dados COSIF para Nubank (via CNPJ da Financeira) ---")
# Esperamos que a classe encontre os dados reportados pelo CNPJ do líder (18236120)
display(dados_nubank_cosif)

In [ ]:
# Objetivo: Ver a situação cadastral e o segmento dos grandes bancos.

bancos = ['BCO DO BRASIL S.A.', 'BRB - BCO DE BRASILIA S.A.', 'CAIXA ECONOMICA FEDERAL']

atributos = ['SITUACAO_IFD_CAD', 'SEGMENTOTB_IFD_CAD']

cadastro_bancos = analisador.get_atributos_cadastro(
    identificador=bancos,
    atributos=atributos
)

print("\n--- Atributos de Cadastro ---")
display(cadastro_bancos)

In [ ]:
# Objetivo: Criar a tabela comparativa que discutimos.

# Definimos os indicadores que queremos na nossa tabela
indicadores_desejados = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMÔNIO LÍQUIDO'},
    'Total Ativos': {'tipo': 'COSIF', 'conta': 'TOTAL GERAL DO ATIVO'},
    # Adicionar o indicador de Basileia do IFDATA aqui quando o método estiver completo
    'Situação': {'tipo': 'Atributo', 'atributo': 'SITUACAO_IFD_CAD'}
}

# Lista de bancos para a comparação
bancos_para_comparar = ['ITAU', 'BRADESCO', 'NUBANK', 'BTG PACTUAL']

# Gerar a tabela
tabela_comparativa = analisador.comparar_indicadores(
    identificadores=bancos_para_comparar,
    indicadores=indicadores_desejados,
    data=202412
)

print("\n--- Tabela Comparativa de Indicadores (Dez/2024) ---")
display(tabela_comparativa)